## YOUGA

### Installation and requirements

In [1]:
# !pip install opencv-python matplotlib numpy mediapipe pyQt5

In [2]:
import math
import cv2
import numpy as np
from time import time
import mediapipe as mp
import matplotlib.pyplot as plt
from PyQt5.QtWidgets import *
from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5 import uic
import time

mp_drawing=mp.solutions.drawing_utils
mp_pose=mp.solutions.pose

In [3]:
def calculateAngle( land1, land2, land3):
    #get landmark coordinates
    x1, y1, _ = land1
    x2, y2, _ = land2
    x3, y3, _ = land3

    angle = math.degrees(math.atan2(y3-y2, x3-x2) - math.atan2(y1-y2, x1-x2))

    if angle<0:
        angle+=360

    return angle
    

In [4]:
def reqAngles(landmarks, *args):
    retList=()
    for arg in args:
        if arg=='le':
            temp = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                      landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value],
                                      landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value])
            
        elif arg=='re':
            temp = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                       landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value],
                                       landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value])
             
        elif arg=='ls':
            temp = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value],
                                         landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                         landmarks[mp_pose.PoseLandmark.LEFT_HIP.value])
        elif arg=='rs':
            temp = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                          landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                          landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value])
        elif arg=='lk':
            temp = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                     landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value],
                                     landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value])
        elif arg=='rk':
            temp = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                      landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value],
                                      landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value])
        elif arg=='lh':
            temp = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                      landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                      landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value])
            
        elif arg=='rh':
            temp = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                      landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                      landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value])
        retList+=(temp,)

    return retList
        
        
            

In [5]:
def correction(minmax, dict):
    probs=[]
    
    if "re" in dict:
        if dict['re']<minmax[0]:
            probs.append("Straighten right elbow a little more")
        elif dict['re']>minmax[1]:
            probs.append("Bend your right elbow more")
        

    if "le" in dict:
        if dict['le']<minmax[2]:
            probs.append("Straighten left elbow a little more")
        elif dict['le']>minmax[3]:
            probs.append("Bend your left elbow more")
        
        

    if "rh" in dict:
        if dict['rh']<minmax[4]:
            probs.append("Straighten your hip")
        elif dict['rh']>minmax[5]:
            probs.append("Bend your hip")
        
    if "lh" in dict:
        if dict['rh']<minmax[6]:
            probs.append("Straighten your hip")
        elif dict['rh']>minmax[7]:
            probs.append("Bend your hip")    
    
    if "rk" in dict:
        if dict['rk']<minmax[8]:
            probs.append("Straighten right your knee")
        elif dict['rk']>minmax[9]:
            probs.append("Bend your right knee")
        
        

    if "lk" in dict:
        if dict['lk']<minmax[10]:
            probs.append("Straighten your left knee")
        elif dict['lk']>minmax[11]:
            probs.append("Bend your left knee")
        
        
        
    if "rs" in dict:
        if dict['rs']<minmax[12]:
            probs.append("Raise your right hand higher")
        elif dict['rs']>minmax[13]:
            probs.append("Lower your right hand")
        
        

    if "ls" in dict:
        if dict['ls']<minmax[14]:
            probs.append("Raise your left hand higher")
        elif dict['ls']>minmax[15]:
            probs.append("Lower your left hand")
        
        
    
    return probs 
    

In [6]:
def tpose(landmarks):
    label='Unknown'
    re, le, rs, ls = reqAngles(landmarks, 're', 'le', 'rs', 'ls')
    
    # ret, let, rst, lst, rkt, lkt, rht, lht= -1, -1, -1, -1, -1, -1, -1, -1  ##Temp variables

    # remin, remax = 150, 200
    # rhmin, rhmax= 60, 120
    # rkmin, rkmax= 150, 200
    # rsmin, rsmax = 150, 200

    minmax= [165, 195, 160, 195, 0, 0, 0, 0, 0, 0, 0, 0, 80, 120, 80, 120]
    corrVal= {}
    probs=[]
    
        # T pose
    if re>minmax[0] and re<minmax[1]:
        pass
    else:
        corrVal['re']=re
    
    if le>minmax[2] and le<minmax[3]:
        pass
    else:
        corrVal['le']=le

    if rs>minmax[12] and rs<minmax[13]:
        pass
    else:
        corrVal['rs']=rs

    if ls>minmax[14] and ls<minmax[15]:
        pass
    else:
        corrVal['ls']=ls
        
    
    if corrVal:
        probs = correction(minmax, corrVal)
    elif not corrVal:
        label= "Tpose"
        probs=[]
    
    return label, probs

def namaskara(landmarks):
    label='Unknown'
    re, le, rk, lk, rs, ls = reqAngles(landmarks, 're', 'le','rk', 'lk', 'rs', 'ls')
    
    # ret, let, rst, lst, rkt, lkt, rht, lht= -1, -1, -1, -1, -1, -1, -1, -1  ##Temp variables

    # remin, remax = 150, 200
    # rhmin, rhmax= 60, 120
    # rkmin, rkmax= 150, 200
    # rsmin, rsmax = 150, 200

    minmax= [35, 75, 295, 335, 0, 0, 0, 0, 150, 200, 150, 200, 0, 40, 0, 40]
    corrVal= {}
    probs=[]
    
        # Namaskarasana
    if re>minmax[0] and re<minmax[1]:
        pass
    else:
        corrVal['re']=re
    
    if le>minmax[2] and le<minmax[3]:
        pass
    else:
        corrVal['le']=le

    if rk>minmax[8] and rk<minmax[9]:
        pass
    else:
        corrVal['rs']=rs

    if lk>minmax[10] and lk<minmax[11]:
        pass
    else:
        corrVal['ls']=ls    

    if rs>minmax[12] and rs<minmax[13]:
        pass
    else:
        corrVal['rs']=rs

    if ls>minmax[14] and ls<minmax[15]:
        pass
    else:
        corrVal['ls']=ls
        
    
    if corrVal:
        probs = correction(minmax, corrVal)
    elif not corrVal:
        label= "Namaskarasana"
        probs=[]
    
    return label, probs


def vriksh(landmarks):
    label='Unknown'
    
    rk, lk = reqAngles(landmarks,'rk', 'lk')


    minmax= [0, 0, 0, 0, 0, 0, 0, 0, 15, 90, 0, 0, 0, 0, 0, 0]
    corrVal= {}
    probs=[]

    if rk>minmax[8] and rk<minmax[9]:
        pass
    else:
        corrVal['rk']=rk

    if corrVal:
        probs = correction(minmax, corrVal)
    elif not corrVal:
        label= "RVrikshasana"
        probs=[]
    
    return label, probs
            
    
            
def veera(landmarks):
    label='Unknown'
    
    re, le, rs, ls, rk, lk, rh, lh = reqAngles(landmarks, 're', 'le', 'rs', 'ls', 'rk', 'lk', 'rh', 'lh')
    
    straightArms=False
    orientation='' #variable to keep track of orientation
    
    if re<195 and re >165  and le<195 and le>160:
        if ls>70 and ls<110 and rs<120 and rs>70:
            ## hands are straight
            straightArms=True
            
    if straightArms:
        ##Hip position check
        if rk>70 and rk<150:  #check if right knee is bent
            if lk>160 and lk<190:
                label="L - VeeraBhadrasana"
            
        elif lk>220 and lk<310:  #check if left knee is bent
            if rk>160 and rk<190:
                label="R - VeeraBhadrasana"
    return label

def vajrasana(landmarks):
    label='Unknown'
    
    rk, rh = reqAngles(landmarks, 'rk', 'rh')

    minmax= [0, 0, 0, 0, 85, 105, 0, 0, 300, 350, 0, 0, 0, 0, 0, 0]
    corrVal= {}
    probs=[]

    if rh>minmax[4] and rh<minmax[5]:
        pass
    else:
        corrVal['rh']=rh

    if rk>minmax[8] and rk<minmax[9]:
        pass
    else:
        corrVal['rk']=rk

    if corrVal:
        probs = correction(minmax, corrVal)
    elif not corrVal:
        label= "R Vajrasana"
        probs=[]
    
    return label, probs

    # if rh>80 and rh<120:  ##check if back is straight
    #     if rk>260 and rk<350:
    #         label ="Right facing Vajrasana"

    # if rh>240 and rh<290:  ##check if back is straight
    #     if rk>0 and rk<40:
    #         label ="Left facing Vajrasana"
    
    return label

def downdog(landmarks):
    label='Unknown'
    re, le, rs, ls, rk, lk, rh, lh = reqAngles(landmarks, 're', 'le', 'rs', 'ls', 'rk', 'lk', 'rh', 'lh')

    # ret, let, rst, lst, rkt, lkt, rht, lht= -1, -1, -1, -1, -1, -1, -1, -1  ##Temp variables

    # remin, remax = 150, 200
    # rhmin, rhmax= 60, 120
    # rkmin, rkmax= 150, 200
    # rsmin, rsmax = 150, 200

    minmax= [150, 200, 0, 0, 60, 120, 0, 0, 50, 200,0, 0, 150, 200, 0, 0]
    corrVal= {}
    probs=[]
    
    ### right facing
    if re>minmax[0] and re<minmax[1]: ##Right facing - elbow
        pass
    else:
        corrVal['re']=re

    if rh>minmax[5] and rh<minmax[6]: ##Right facing - hip
        pass
    else:
        corrVal['rh']=rh
    
    if rk>minmax[8] and rk<minmax[9]:
        pass
    else:
        corrVal['rk']=rk
    
    if rs>minmax[12] and rs<minmax[13]:
        pass
    else:
        corrVal['rs']=rs
    
    
    if corrVal:
        probs = correction(minmax, corrVal)
    elif not corrVal:
        label= "Downward dog"
        probs=[]
    
    return label, probs



In [7]:
def classifyPose(landmarks, output_image, option, display=False):
    label = 'Unknown'
    problems = ["Perfect! Youre great at this!"]
    labelColor = (0, 0, 255)
    problemColor = (0, 255, 255)

    '''
    re : right elbow
    le : left elbow
    
    rs : right shoulder
    ls : left shoulder

    rk : right knee
    lk : left knee

    rh : right hip
    lh : left hip

    '''

    rightPose=None
    match option:
        case 1:
            label, problems = tpose(landmarks)
            rightPose="TPose"
        case 2:
            label, problems = vriksh(landmarks)
            rightPose="Vrikshasana"
        case 3:
            label = veera(landmarks)
            rightPose="Veerabhadrasana"
        case 4:
            label, problems = vajrasana(landmarks)
            rightPose="Vajrasana"       
        case 5:
            label, problems = namaskara(landmarks)
            rightPose="Namaskara"
        case 6:
            label, problems = downdog(landmarks)
            rightPose="Downward-Dog"

    
    if label != 'Unknown':
        labelColor = (0, 255, 0)

    ## Print name of asana
    cv2.putText(output_image, label, (10, 30), cv2.FONT_HERSHEY_PLAIN, 2, labelColor, 2)
    
    flag=False
    if not problems:
        problems=["Perfect!!!"]
        flag=True
    #     problemColor = (255, 255, 0)

    # count=0
    # for i in problems:
    #     count+=25
    ##Print problems
        # cv2.putText(output_image, i, (10, 100+count), cv2.FONT_HERSHEY_PLAIN, 1, problemColor, 2)

    if display:

        plt.figure(figsize=[10, 10])
        plt.imshow(output_image[:, :, ::-1]);plt.title("Output image");plt.axis('off');

    else:
        return output_image, label, problems, rightPose, flag

In [8]:
def detectPose(image, pose, display=True):
    '''
    This function performs pose detection on an image.
    Args:
        image: The input image with a prominent person whose pose landmarks needs to be detected.
        pose: The pose setup function required to perform the pose detection.
        display: A boolean value that is if set to true the function displays the original input image, the resultant image, 
                 and the pose landmarks in 3D plot and returns nothing.
    Returns:
        output_image: The input image with the detected pose landmarks drawn.
        landmarks: A list of detected landmarks converted into their original scale.
    '''
    
    # Create a copy of the input image.
    output_image = image.copy()
    
    # Convert the image from BGR into RGB format.
    imageRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Perform the Pose Detection.
    results = pose.process(imageRGB)
    
    # Retrieve the height and width of the input image.
    height, width, _ = image.shape
    
    # Initialize a list to store the detected landmarks.
    landmarks = []
    
    # Check if any landmarks are detected.
    if results.pose_landmarks:
    
        # Draw Pose landmarks on the output image.
        mp_drawing.draw_landmarks(image=output_image, landmark_list=results.pose_landmarks,
                                  connections=mp_pose.POSE_CONNECTIONS)
        
        # Iterate over the detected landmarks.
        for landmark in results.pose_landmarks.landmark:
            
            # Append the landmark into the list.
            landmarks.append((int(landmark.x * width), int(landmark.y * height),
                                  (landmark.z * width)))
    
    ### Check if the original input image and the resultant image are specified to be displayed.
    if display:
    
        # Display the original input image and the resultant image.
        plt.figure(figsize=[22,22])
        plt.subplot(121);plt.imshow(image[:,:,::-1]);plt.title("Original Image");plt.axis('off');
        plt.subplot(122);plt.imshow(output_image[:,:,::-1]);plt.title("Output Image");plt.axis('off');
        
        # Also Plot the Pose landmarks in 3D.
        mp_drawing.plot_landmarks(results.pose_world_landmarks, mp_pose.POSE_CONNECTIONS)
        
    # Otherwise
    else:
        
        # Return the output image and the found landmarks.
        return output_image, landmarks

In [9]:
# mp_drawing=mp.solutions.drawing_utils
# mp_pose=mp.solutions.pose

#     #Video tracking
# pose_vid = mp_pose.Pose(min_detection_confidence=0.3, min_tracking_confidence=0.5)

#     #Video port
# cap=cv2.VideoCapture(0)
# cap.set(3, 1280)
# cap.set(4, 960)

# cv2.namedWindow('Youga', cv2.WINDOW_NORMAL)

# option='e' ## Option for pose

# while cap.isOpened():
#     ret, frame = cap.read()
            
#     if not ret:
#         continue
#     frame=cv2. flip(frame, 1)
#     frame_height, frame_width, _ = frame.shape
#     frame=cv2.resize(frame, (int(frame_width*(640/frame_height)), 640))

#     frame, landmarks=detectPose(frame, pose_vid, display=False)

#     if landmarks:
#         frame, _=classifyPose(landmarks, frame, option, display=False)
            
#     cv2.imshow("Youga feed", frame)
            
#     if cv2.waitKey(10) & 0xFF == ord('q'):
#         break
        
# cap.release()
# cv2.destroyAllWindows()

In [10]:
class GUI(QMainWindow):

    def __init__(self):
        super(GUI, self).__init__()
        uic.loadUi("Frontend/DetectionAndCorrection.ui", self)

        self.setStyleSheet("background-color: rgba(0, 28, 41, 255);")

        self.exitButton.clicked.connect(self.stopRecording)

        self.nextButton.clicked.connect(self.changeClassOption)

        #initialize the worker thread
        self.Worker1=Worker1(1)
        self.Worker1.start()
        self.Worker1.ImageUpdate.connect(self.ImageUpdateSlot)
        self.count=0

        # self.result.setAlignment(Qt.AlignCenter)

        font = QFont()
        font.setPointSize(18)

        self.result.setFont(font)
        # self.yogaPose.setStyleSheet("background-image: url('/home/giridhar/Desktop/Youga/images/cobrapose.jpg');background-size: cover;")
        self.textEdit.setFont(font)

        
        bg_image_initial = cv2.imread("/home/giridhar/Desktop/Youga/images/1.jpeg")
        bg_image = cv2.cvtColor(bg_image_initial, cv2.COLOR_BGR2RGB)
        ConvertToQtFormat=QImage(bg_image.data, bg_image.shape[1], bg_image.shape[0], QImage.Format_RGB888)
        bg=ConvertToQtFormat.scaled(360, 240, Qt.KeepAspectRatio)
        self.yogaPose.setPixmap(QPixmap.fromImage(bg))

        # Set text color
        self.setStyleSheet("color: white;")

        self.show()
    
    def ImageUpdateSlot(self, image, probs, label, flag):
        self.cameraFeed.setPixmap(QPixmap.fromImage(image))
        self.textEdit.setPlainText(probs.toPlainText())
        self.result.setPlainText(label.toPlainText())

        self.result.setStyleSheet(self.result.styleSheet() + 'color: black;')

        # if flag:
        #     time.sleep(1)
        #     self.count+=1
        # else:
        #     self.count=0

        if self.count==3:
            self.changeClassOption()


    def changeClassOption(self):
        self.Worker1.classOption=self.Worker1.classOption+1
        if self.Worker1.classOption>5:
            self.Worker1.classOption=1
        print("Class option changed to:", self.Worker1.classOption)

        bg_image_initial = cv2.imread("/home/giridhar/Desktop/Youga/images/"+str(self.Worker1.classOption)+".jpg")
        bg_image = cv2.cvtColor(bg_image_initial, cv2.COLOR_BGR2RGB)
        ConvertToQtFormat=QImage(bg_image.data, bg_image.shape[1], bg_image.shape[0], QImage.Format_RGB888)
        bg=ConvertToQtFormat.scaled(360, 240, Qt.KeepAspectRatio)
        self.yogaPose.setPixmap(QPixmap.fromImage(bg))

    def stopRecording(self):
        print("Exit Entered")
        self.Worker1.exit()

    

    

class Worker1(QThread):
    ImageUpdate=pyqtSignal(QImage, QTextDocument, QTextDocument, bool)

    def __init__(self, classOption, parent=None):
        super().__init__(parent)
        self.classOption = classOption

    def run(self):
        self.ThreadActive= True
        mp_drawing=mp.solutions.drawing_utils
        mp_pose=mp.solutions.pose

        #Video tracking
        pose_vid = mp_pose.Pose(min_detection_confidence=0.3, min_tracking_confidence=0.5)

        #Video port
        try:
            cap=cv2.VideoCapture(0)
            cap.set(3, 1280)
            cap.set(4, 960)

            # cv2.namedWindow('Youga', cv2.WINDOW_NORMAL)

            ## Option for pose

            while self.ThreadActive:
                try:
                    option=self.classOption
                    ret, frame = cap.read()
                    
                    poseCorrect=False
                    if not ret:
                        continue
                    frame=cv2. flip(frame, 1)
                    frame_height, frame_width, _ = frame.shape
                    frame=cv2.resize(frame, (int(frame_width*(480/frame_height)), 480))
                    # frame=cv2.resize(frame, 640, 480)

                    landmarks=[1]
                    
                    frame, landmarks=detectPose(frame, pose_vid, display=False)

                    if landmarks:
                        frame, label, problems, poseName, flag=classifyPose(landmarks, frame, option, display=False)
                    
                    if flag:
                        poseCorrect=True
                    # cv2.imshow("Youga feed", frame)
                    frameColorCorrected = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    ConvertToQtFormat=QImage(frameColorCorrected.data, frameColorCorrected.shape[1], frameColorCorrected.shape[0], QImage.Format_RGB888)
                    pic=ConvertToQtFormat.scaled(640, 480, Qt.KeepAspectRatio)

                    text_block = QTextDocument()
                    probs="\n".join(problems)
                    text_block.setPlainText(probs)

                    text_block2 = QTextDocument()
                    text_block2.setPlainText(poseName)
                    self.ImageUpdate.emit(pic, text_block, text_block2, poseCorrect)
                except:
                    pass
        except:
            print("ISSUE!!")        
    def exit(self):        
        self.ThreadActive=False
        self.quit()
                

def main():
    app=QApplication([])
    window=GUI()
    app.exec_()


if __name__=='__main__':
    main()

QSocketNotifier: Can only be used with threads started with QThread
I0000 00:00:1713380882.519410    5830 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1713380882.526518    5844 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.0.4-0ubuntu1~22.04.1), renderer: AMD Radeon Vega 8 Graphics (raven, LLVM 15.0.7, DRM 3.54, 6.5.0-26-generic)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Class option changed to: 2
Class option changed to: 3
Class option changed to: 4
Class option changed to: 5
Class option changed to: 1
Class option changed to: 2
Class option changed to: 3
Class option changed to: 4
Class option changed to: 5
Class option changed to: 1
Class option changed to: 2
Class option changed to: 3


: 